In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [2]:
# Read the text file
with open('/content/drive/MyDrive/sherlock-holm.es_stories_plain-text_advs.txt', 'r', encoding='utf-8') as file:
    text = file.read()

In [3]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

In [4]:
input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [5]:
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [6]:
X = input_sequences[:, :-1]
y = input_sequences[:, -1]

In [7]:
y = np.array(tf.keras.utils.to_categorical(y, num_classes=total_words))

In [8]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 17, 100)           820000    
                                                                 
 lstm (LSTM)                 (None, 150)               150600    
                                                                 
 dense (Dense)               (None, 8200)              1238200   
                                                                 
Total params: 2208800 (8.43 MB)
Trainable params: 2208800 (8.43 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [9]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=20, verbose=1)

Epoch 1/20
3010/3010 [==============================] - 208s 68ms/step - loss: 6.2474 - accuracy: 0.0763
Epoch 2/20
3010/3010 [==============================] - 200s 67ms/step - loss: 5.5176 - accuracy: 0.1223
Epoch 3/20
3010/3010 [==============================] - 199s 66ms/step - loss: 5.1286 - accuracy: 0.1470
Epoch 4/20
3010/3010 [==============================] - 195s 65ms/step - loss: 4.8047 - accuracy: 0.1634
Epoch 5/20
3010/3010 [==============================] - 192s 64ms/step - loss: 4.5050 - accuracy: 0.1824
Epoch 6/20
3010/3010 [==============================] - 193s 64ms/step - loss: 4.2230 - accuracy: 0.2011
Epoch 7/20
3010/3010 [==============================] - 190s 63ms/step - loss: 3.9493 - accuracy: 0.2258
Epoch 8/20
3010/3010 [==============================] - 194s 64ms/step - loss: 3.6916 - accuracy: 0.2564
Epoch 9/20
3010/3010 [==============================] - 191s 63ms/step - loss: 3.4486 - accuracy: 0.2902
Epoch 10/20
3010/3010 [==============================] 

In [10]:
seed_text = "I will leave if they"
next_words = 5

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=-1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word

print(seed_text)

1/1 [==============================] - 0s 47ms/step
I will leave if they were not on the day
